In [1]:
# https://github.com/pytorch/pytorch/issues/137695#issuecomment-2412944849
!ulimit -n 131072

In [2]:
!python vllm/collect_env.py

WARNING 10-16 00:02:24 rocm.py:13] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
PyTorch version: 2.6.0.dev20241015+rocm6.2
Is debug build: False
CUDA used to build PyTorch: N/A
ROCM used to build PyTorch: 6.2.41133-dd7f95766

OS: Ubuntu 22.04.5 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: version 3.30.5
Libc version: glibc-2.35

Python version: 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0] (64-bit runtime)
Python platform: Linux-6.8.0-45-generic-x86_64-with-glibc2.35
Is CUDA available: True
CUDA runtime version: Could not collect
CUDA_MODULE_LOADING set to: LAZY
GPU models and configuration: AMD Instinct MI300X (gfx942:sramecc+:xnack-)
Nvidia driver version: Could not collect
cuDNN version: Could not collect
HIP runtime version: 6.2.41133
MIOpen runtime version: 3.2.0
Is XNNPACK available: True

CPU:
Architecture:                 

In [3]:
# Triton FA
!VLLM_USE_TRITON_FLASH_ATTN=1 python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 512 --output-len 128 --model meta-llama/Llama-2-7b-chat-hf -tp 8

WARNING 10-16 00:02:29 rocm.py:13] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=512, output_len=128, model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-16 00:02:39 config.py:887] Defaulting to use mp for distributed inference
INFO 10-16 00:02:39 config.py:

In [4]:
# CK FA
!VLLM_USE_TRITON_FLASH_ATTN=0 python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len 512 --output-len 128 --model meta-llama/Llama-2-7b-chat-hf -tp 8

WARNING 10-16 00:04:01 rocm.py:13] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
Namespace(backend='vllm', dataset=None, input_len=512, output_len=128, model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', quantization=None, tensor_parallel_size=8, n=1, num_prompts=1000, seed=0, hf_max_batch_size=None, trust_remote_code=False, max_model_len=None, dtype='auto', gpu_memory_utilization=0.9, enforce_eager=False, kv_cache_dtype='auto', quantization_param_path=None, device='auto', num_scheduler_steps=1, use_v2_block_manager=True, enable_prefix_caching=False, enable_chunked_prefill=False, max_num_batched_tokens=None, download_dir=None, output_json=None, distributed_executor_backend=None, load_format='auto', disable_async_output_proc=False, async_engine=False, disable_frontend_multiprocessing=False)
INFO 10-16 00:04:13 config.py:887] Defaulting to use mp for distributed inference
INFO 10-16 00:04:13 config.py:

In [5]:
!pip install nbformat

DEPRECATION: Loading egg at /mnt/nvme1n1p1/miniforge3/envs/vllm/lib/python3.11/site-packages/vllm-0.6.4.dev9+g5d264f4a.rocm624-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /mnt/nvme1n1p1/miniforge3/envs/vllm/lib/python3.11/site-packages/flash_attn-2.6.3-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
import pandas as pd
import re
import nbformat

triton_fa = 'Throughput: 52.43 requests/s, 33556.50 tokens/s'
ck_fa = 'Throughput: 55.22 requests/s, 35340.21 tokens/s'

# Use regular expressions to extract numbers
matches = re.findall(r"Throughput: ([\d.]+) requests/s, ([\d.]+) tokens/s", triton_fa)
if matches:
    requests_per_sec, tokens_per_sec = matches[0]
    data = {
        'FA': 'triton',
        'Requests per Second': float(requests_per_sec),
        'Tokens per Second': float(tokens_per_sec)
    }
    df = pd.DataFrame([data])

matches = re.findall(r"Throughput: ([\d.]+) requests/s, ([\d.]+) tokens/s", ck_fa)
if matches:
    requests_per_sec, tokens_per_sec = matches[0]
    data = {
        'FA': 'ck', 
        'Requests per Second': float(requests_per_sec),
        'Tokens per Second': float(tokens_per_sec)
    }
    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

print(df)

       FA  Requests per Second  Tokens per Second
0  triton                52.43           33556.50
1      ck                55.22           35340.21


In [7]:
# Extract values for 'ck'
ck_rps = df.loc[df['FA'] == 'ck', 'Requests per Second'].values[0]
ck_tps = df.loc[df['FA'] == 'ck', 'Tokens per Second'].values[0]

# Extract values for 'triton'
triton_rps = df.loc[df['FA'] == 'triton', 'Requests per Second'].values[0]
triton_tps = df.loc[df['FA'] == 'triton', 'Tokens per Second'].values[0]

# Calculate percentage difference for Requests per Second (RPS)
percent_diff_rps = ((ck_rps - triton_rps) / triton_rps) * 100
print(percent_diff_rps, 'Requests per Second faster')

# Calculate percentage difference for Tokens per Second (TKS)
percent_diff_tps = ((ck_tps - triton_tps) / triton_tps) * 100
print(percent_diff_tps, 'Tokens per Second faster')

5.321380888804119 Requests per Second faster
5.3155424433418235 Tokens per Second faster


In [8]:
def benchmark_model(model, input_len, output_len, tp):
    import pandas as pd
    import re
    
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['FA', 'Requests per Second', 'Tokens per Second'])
    
    # Function to run the benchmark command and capture output
    def run_benchmark(use_triton):
        # Set the environment variable
        VLLM_USE_TRITON_FLASH_ATTN = '1' if use_triton else '0'
        # Construct the command
        command = f"VLLM_USE_TRITON_FLASH_ATTN={VLLM_USE_TRITON_FLASH_ATTN} python vllm/benchmarks/benchmark_throughput.py --backend vllm --input-len {input_len} --output-len {output_len} --model {model} -tp {tp}"
        # Run the command and capture the output
        output = get_ipython().getoutput(command)
        output_str = ' '.join(output)
        # Use regular expressions to extract the throughput values
        matches = re.findall(r"Throughput:\s*([\d.]+)\s*requests/s,\s*([\d.]+)\s*tokens/s", output_str)
        if matches:
            requests_per_sec, tokens_per_sec = map(float, matches[0])
            return requests_per_sec, tokens_per_sec
        else:
            print(f"No throughput data found for {'Triton' if use_triton else 'CK'} FA.")
            return None, None

    # Run benchmarks for CK FA (use_triton=False)
    ck_rps, ck_tps = run_benchmark(use_triton=False)
    if ck_rps is None or ck_tps is None:
        print("Benchmark failed for CK FA.")
        return None

    # Append CK FA results to the DataFrame
    df.loc[len(df)] = {'FA': 'ck', 'Requests per Second': ck_rps, 'Tokens per Second': ck_tps}

    # Run benchmarks for Triton FA (use_triton=True)
    triton_rps, triton_tps = run_benchmark(use_triton=True)
    if triton_rps is None or triton_tps is None:
        print("Benchmark failed for Triton FA.")
        return None

    # Append Triton FA results to the DataFrame
    df.loc[len(df)] = {'FA': 'triton', 'Requests per Second': triton_rps, 'Tokens per Second': triton_tps}

    # Calculate percentage differences (Triton is baseline)
    percent_diff_rps = ((ck_rps - triton_rps) / triton_rps) * 100
    percent_diff_tps = ((ck_tps - triton_tps) / triton_tps) * 100
    avg_percent_diff = (percent_diff_rps + percent_diff_tps) / 2

    # Add percentage differences to the DataFrame
    df['% Difference RPS'] = [percent_diff_rps, 0]
    df['% Difference TPS'] = [percent_diff_tps, 0]
    df['% Difference Avg'] = [avg_percent_diff, 0]

    # Display the DataFrame
    print(df)
    return df

In [9]:
# Call the function with your parameters
df_results = benchmark_model(
    model='meta-llama/Llama-2-7b-chat-hf',
    input_len=128,
    output_len=128,
    tp=8
)
display(df_results)

       FA  Requests per Second  Tokens per Second  % Difference RPS  \
0      ck                83.72           21432.81          7.984006   
1  triton                77.53           19847.90          0.000000   

   % Difference TPS  % Difference Avg  
0          7.985278          7.984642  
1          0.000000          0.000000  


,FA,Requests per Second,Tokens per Second,% Difference RPS,% Difference TPS,% Difference Avg
0,ck,83.72,21432.81,7.984006,7.985278,7.984642
1,triton,77.53,19847.90,0.000000,0.000000,0.000000


In [10]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 256, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 512, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 1024, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 2048, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 0, 'output_len': 4096, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-2-7b-chat-hf', 'input_len': 0, 'output_len': 128, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Difference RPS  \
0      ck                93.64           11985.92          7.867757   
1  triton                86.81           11111.56          0.000000   

   % Difference TPS  % Difference Avg  
0          7.868922           7.86834  
1          0.000000           0.00000  
{'model': 'meta-llama/Llama-2-7b-chat-hf', 'input_len': 0, 'output_len': 256, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Difference RPS  \
0      ck                48.52           12422.23         10.978957   
1  triton                43.72           11191.04          0.000000   

   % Difference TPS  % Difference Avg  
0         11.001569         10.990263  
1          0.000000          0.000000  
{'model': 'meta-llama/Llama-2-7b-chat-hf', 'input_len': 0, 'output_len': 512, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Difference RPS  \


,FA,Requests per Second,Tokens per Second,% Difference RPS,% Difference TPS,% Difference Avg,Config
0,ck,93.64,11985.92,7.867757,7.868922,7.868340,"input_len=0, output_len=128, tp=8"
1,triton,86.81,11111.56,0.000000,0.000000,0.000000,"input_len=0, output_len=128, tp=8"
2,ck,48.52,12422.23,10.978957,11.001569,10.990263,"input_len=0, output_len=256, tp=8"
3,triton,43.72,11191.04,0.000000,0.000000,0.000000,"input_len=0, output_len=256, tp=8"
4,ck,22.56,11548.59,-1.009215,-1.045787,-1.027501,"input_len=0, output_len=512, tp=8"
5,triton,22.79,11670.64,0.000000,0.000000,0.000000,"input_len=0, output_len=512, tp=8"
6,ck,10.82,11077.85,1.787394,1.740205,1.763800,"input_len=0, output_len=1024, tp=8"
7,triton,10.63,10888.37,0.000000,0.000000,0.000000,"input_len=0, output_len=1024, tp=8"
8,ck,4.80,9828.08,-0.207900,-0.170850,-0.189375,"input_len=0, output_len=2048, tp=8"
9,triton,4.81,9844.90,0.000000,0.000000,0.000000,"input_len=0, output_len=2048, tp=8"


In [11]:
# List of configurations to test
configs = [
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 1024, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 2048, 'output_len': 128, 'tp': 8},
    {'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 4096, 'output_len': 128, 'tp': 8},
]

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Run benchmarks for each configuration
for config in configs:
    print(config)
    df_result = benchmark_model(**config)
    if df_result is not None:
        # Add a column for the configuration
        df_result['Config'] = f"input_len={config['input_len']}, output_len={config['output_len']}, tp={config['tp']}"
        all_results = pd.concat([all_results, df_result], ignore_index=True)

# Display all results
display(all_results)

{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 128, 'output_len': 128, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Difference RPS  \
0      ck                84.56           21647.27         24.536082   
1  triton                67.90           17381.82          0.000000   

   % Difference TPS  % Difference Avg  
0          24.53972         24.537901  
1           0.00000          0.000000  
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 256, 'output_len': 128, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Difference RPS  \
0      ck                75.69           29065.78         18.934632   
1  triton                63.64           24437.42          0.000000   

   % Difference TPS  % Difference Avg  
0         18.939643         18.937137  
1          0.000000          0.000000  
{'model': 'meta-llama/Llama-3.1-8B-Instruct', 'input_len': 512, 'output_len': 128, 'tp': 8}
       FA  Requests per Second  Tokens per Second  % Dif

,FA,Requests per Second,Tokens per Second,% Difference RPS,% Difference TPS,% Difference Avg,Config
0,ck,84.56,21647.27,24.536082,24.539720,24.537901,"input_len=128, output_len=128, tp=8"
1,triton,67.90,17381.82,0.000000,0.000000,0.000000,"input_len=128, output_len=128, tp=8"
2,ck,75.69,29065.78,18.934632,18.939643,18.937137,"input_len=256, output_len=128, tp=8"
3,triton,63.64,24437.42,0.000000,0.000000,0.000000,"input_len=256, output_len=128, tp=8"
4,ck,63.53,40659.06,3.116377,3.111760,3.114068,"input_len=512, output_len=128, tp=8"
5,triton,61.61,39432.03,0.000000,0.000000,0.000000,"input_len=512, output_len=128, tp=8"
6,ck,46.86,53986.25,10.414703,10.417384,10.416043,"input_len=1024, output_len=128, tp=8"
7,triton,42.44,48892.89,0.000000,0.000000,0.000000,"input_len=1024, output_len=128, tp=8"
8,ck,31.33,68172.61,8.483380,8.477709,8.480544,"input_len=2048, output_len=128, tp=8"
9,triton,28.88,62844.81,0.000000,0.000000,0.000000,"input_len=2048, output_len=128, tp=8"
